# Word Overlap

Today, we're going to implement a word overlap counter with tf-idf scaling! We're going to add the results to our feature vector. We're also going to begin filling in our final code.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import csv
import os
import re
from nltk.stem import WordNetLemmatizer
from math import log

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
train_body_path = "train_bodies.csv"
if not os.path.exists(train_body_path):
    print("Check location for train_bodies")
test_body_path = "test_bodies.csv"
if not os.path.exists(test_body_path):
    print("Check location for test_bodies")
train_stances_path = "train_stances.csv"
if not os.path.exists(train_stance_path):
    print("Check location for train_stances")
test_headline_path = "test_stances_unlabeled.csv"
if not os.path.exists(test_headline_path):
    print("Check location for test_stances_unlabeled")

## Getting started

As you guys know, this code has a lot of parts. Normally, when you work on a large project like this, you would leave it in one (or several!) files and come back to it. Instead, we're going to copy our project to each day's notebook, adding more and more pieces as we complete them. First, we have to redefine the functions we've already written, so that our code knows where to find them. You can run these all as-is -- the code is just copied from what you've already written --- but definitely read them over and ask if you have any questions.

In [ ]:
# Here's the code we've written so far!
# You don't have to do anything, but read it over and make sure you remember what each function is doing

def clean(s):
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

def w_tokenize(s):
    return nltk.word_tokenize(s)

def s_tokenize(p):
    return nltk.sent_tokenize(p)

def lemmatize(word_tokens):
    return [lemmatizer.lemmatize(t) for t in word_tokens]

def remove_stopwords(word_tokens):
    return [w for w in word_tokens if not w in stop_words]

def w_super_clean(s):
    return remove_stopwords(lemmatize(w_tokenize(clean(s))))

def s_super_clean(p):
    sentences = s_tokenize(p)
    clean_sentences = []
    for s in sentences:
        clean_sentences.append(" ".join(remove_stopwords(lemmatize(w_tokenize(clean(s))))))
    return clean_sentences

In [ ]:
# Here's our load body function from before
# Again, you don't need to do anything, but read through and ask if any lines confuse you
def load_body(filename):
    id2body = {} 
    id2body_sentences = {} 
    
    # These lines open the file and read in each row
    with open(filename, encoding='utf-8', errors='ignore') as fh:
        reader = csv.DictReader(fh)
        data = list(reader)
        for row in data:
            
            # This line gets the Body ID for this row
            id = row['Body ID']
            # This line gets the article body
            body = str(row['articleBody'])
            # This line strips leading and trailing spaces from the body
            body = body.strip()
            
            # Cleaning words and sentences
            body_words = w_super_clean(body) 
            body_sentences = s_super_clean(body)
            
            # Adding to the two dictionaries
            id2body[id] = body_words
            id2body_sentences[id] = body_sentences
    
    return id2body, id2body_sentences

# Load in headlines - body pairs and their stances for training
def load_stance(filename):
    stances = []
    
    with open(filename, encoding='utf-8', errors='ignore') as fh:
        reader = csv.DictReader(fh)
        data = list(reader)
        for row in data:
            clean_title = w_super_clean(row['Headline'])
            stances.append((clean_title, row['Body ID'], row['Stance'].strip()))
    return stances

In [ ]:
# Our counting dictionary function
def dictionary_count(count_items, count_dictionary):
    for item in count_items:        
        if item in count_dictionary:
            count_dictionary[item] += 1
        else: 
            count_dictionary[item] = 1
    
    return count_dictionary

# Our duplicate-eliminating function
def elim_dupes(items):
    new_list = []
    for item in items:
        if item not in new_list:
            new_list.append(item)
    return new_list

# Our function for making the idf!
def prepare_idf(corpus):
    docs_containing = {}
    idf = {}
    for (body_id, body) in corpus.items():
        no_dupes = elim_dupes(body)
        docs_containing = dictionary_count(no_dupes, docs_containing)
    for word in docs_containing:
        idf[word] = log(len(corpus) / docs_containing[word])
    return idf

In [ ]:
# We have to run this once so we can test our word overlap function
id2body, id2body_sentences = load_body(train_body_path) 
idf = prepare_idf(id2body)
train_stances = load_stance(train_stances_path)

## Calculating overlap

Alright! The next step in our feature extraction is to write our full function to get word overlap. We're going to write a function that takes in a title-body pair, as well as our idf, and returns the overlap between the two. This way, we can call it every time we loop through a new pair. 

We're going to return a list with four pieces. The first will be the idf-scaled overlap. The second, the idf-scaled overlap over the maximum possible idf-scaled overlap (will be between 1 and 0). The third item is a raw count of overlapping words (no idf scaling). The final component is a count of overlapping words divided by the maximum possible count of overlapping words (will be between 1 and 0). 

Feel free to ask if you have any questions about this at all!

In [ ]:
# This function returns an array representing the overlap between title and body
def get_word_overlaps(title, body, idf):
    words_in_body = {}
    words_in_title = {}
    
    #TODO: use dictionary_count to count words in body
    words_in_body = _________
    #TODO: use dictionary_count to count words in title
    words_in_title = __________

    maximum_scaled = 0.0
    # TODO: get the maximum word overlap count by taking the length of the title
    maximum_count = ____________
    
    for (word, title_word_count) in words_in_title.items():
        # TODO: calculate maximum possible scaled overlap by multiplying the count of each word times its idf
        maximum_scaled += 

    overlap_scaled = 0
    overlap_count = 0
    
    for (word, title_word_count) in words_in_title.items():
        # TODO: check if this word is in the body
        if ______ :
            # TODO: get the number of overlaps by finding the minumum between how many times it appears in the title and the body
            tf = ___________
            
            # TODO: add the number of overlaps to the overlap count
            ___________________
            # TODO: scale the number of overlaps by the idf of the word and add it to the scaled overlap count
            ____________________

    # TODO: divide the scaled overlap by the maximum possible scaled overlap
    scaled_over_max = _____________
    # TODO: divide the overlap count by the maximum possible overlap count
    count_over_max = _____________
    
    # TODO: return a vector of overlap_count, count_over_max, overlap_scaled, and scaled_over_max
    return [_____________]

To test this out, let's make a rudimentary version of our extract_features function that we can call on a headline-article pair. 

In [ ]:
def extract_features(title, body, idf):
    # TODO: make an empty list to hold the features
    features = _____________
    
    # TODO: get the word overlap and append it to the feature list
    _____________
    
    # TODO: return the feature vector
    _____________

Let's test out extract features! We'll pair headlines and bodies from the first five examples. Do the respective feature vectors look right to you?

In [ ]:
mini_stances = train_stances[:5]
for (title, body_id, stance) in mini_stances:
        body = id2body[body_id]
        features = extract_features(title, body, idf)
        print(title)
        print(body[:25])
        print(features)
        print("------------------------------------------------------")

## Filling in the skeleton

Below is the code skeleton from yesterday. Today, all the pieces that you have will include "TODO" in front of them. Where you see TODO, fill in a line (or lines) below in order to complete the task that the comments describe. Comments that begin with "Later", we haven't completed the code for yet. You can leave those blank. 

In [ ]:
def make_predictions():
    
    # TODO: Load and clean the training body --- remember, this function returns two values
    id2body, id2body_sentences = _____________
    
    # TODO: Prepare the idf
    idf = _____________
    
    # TODO: Load and clean the headline-article-stance sets for training using the load_stances function near load_body
    train_stances = _____________
    
    # Later: Make a predictor to train on the labeled pairs and predict on the unlabeled pairs
    
    # TODO: Loop through every item in the stance dataset
    _____________:
        # TODO: Get the article body that corresponds to this body_id from id2body
        body = _____________
        # TODO: Call the extract_features function
        features = _____________
        
        # Later: Do one training step for a predictor using those features and the correct label
    
    # Later: Load and clean the headlines (test set) 
    
    # Later: For every example in the test set:
        # Later: Extract the features 
        # Later: Use the predictor to make a prediction based on those features
    
    # Later: Check our predicted answers against the real answers
    # Later: Output accuracy measures!
    

We're not going to run this today, but we want to begin piecing our code together! It will make it easier to assemble in the coming days. 

## Challenge 1: Trimmed Overlap

Right now, we're checking the word overlap between the title and the full body. One other piece of information that might be useful in a feature vector is how much word overlap there is between the title and the first few sentences of the article. News pieces tend start out with a short summary of the important information in the article, so articles that begin with a lot of similar words to the title might be more likely to be directly talking about those words, rather than if they appear somewhere in the middle, or spread across at random. 

Let's write a function to compute the overlap between the title and a trimmed version of the body. It's hard to know where to trim --- after three sentences? At a paragraph break? To keep the ratios fair, however, we should scale that length according to the length of the title. If we check a short title against a long article and a long title against a long article, which is more likely to have a high overlap percentage?

What we're going to do is trim the article text at 4x the length of the title --- if the title is ten words, we'll do an overlap test with the first 40 words of the article body. Read about list slicing here:

https://www.pythoncentral.io/how-to-slice-listsarrays-and-tuples-in-python/

And see if you can figure out how to modify your word_overlaps function to slice the article body to length before checking its overlaps. 

In [ ]:
# Alright! We're going to compute the overlap
def get_word_overlaps_trimmed(title, body, idf):
    # TODO: paste your code for getting word overlaps here. 
    
    # TODO: figure out how to slice the list of article words to be four times the length of the headline
    

We'll also have to update our feature extraction function! You can copy-paste your previous solution and modify it.

In [ ]:
def extract_features(title, body, idf):
    features = []
    
    # TODO: update features with get_word_overlaps with full article body
    
    # TODO: update features with get_word_overlaps_trimmed with trimmed article body
    
    return features

In [ ]:
# Testing feature extraction. Check that your list is eight numbers long, and that the values make sense.
mini_stances = train_stances[:5]
for (title, body_id, stance) in mini_stances:
        body = id2body[body_id]
        features = extract_features(title, body, idf)
        print(title)
        print(body[:25])
        print(features)
        print("------------------------------------------------------")

## Challenge 2: Good Programming Practice

You may have noticed that the code you just wrote was preeeetty similar to the code you wrote for the initial get_word_overlaps function. This happens pretty often --- you notice that something you're writing feels mighty familiar, and then you go back and realize that you might've been able to use something like the code you've already written. Of course, copy-paste is always an option (ON YOUR OWN CODE), but the better way to handle such things is usually to figure out how to change the code into a function that works for both scenarios. Sometimes this means introducing another variable, or changing up the structure a little --- that's okay! Even if writing the code is a little more challenging, you'll end up with cleaner code than if you repeated big chunks. 

For this challenge, we're going to try to write a version of the get_word_overlap function that can be used to get overlaps in a full version of the body or a trimmed one. We're going to do this by introducing another argument, that tells us how much of the body we want to include. 



In [ ]:
# Alright! We're going to compute the overlap
def get_word_overlaps_flex(title, body, idf, body_len):

    # TODO: paste your code for getting word overlaps here. 
    
    # TODO: figure out how to modify the code so that it slices to body_len length (similar to above)

We'll have to update our feature extraction function again! You can copy-paste your previous solution and modify it.

In [ ]:
def extract_features(title, body, idf):
    features = []
    
    # TODO: update features with get_word_overlaps_flex with full article body
    
    # TODO: update features with get_word_overlaps_flex with trimmed article body
    
    return features

In [ ]:
# Testing feature extraction. Check that your list here looks like 
mini_stances = train_stances[:5]
for (title, body_id, stance) in mini_stances:
        body = id2body[body_id]
        features = extract_features(title, body, idf)
        print(title)
        print(body[:25])
        print(features)
        print("------------------------------------------------------")